In [1]:
# Matplotlib forms basis for visualization in Python
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

# We will use the Seaborn library
import seaborn as sns
import plotly.express as px

sns.set()

# Graphics in SVG format are more sharp and legible
%config InlineBackend.figure_format = 'svg'

In [23]:
raw_df = pd.read_excel('./data/presupuesto/gastos.xlsx', index_col='Departamento')

In [24]:
def clean(raw_df):
    raw_df = raw_df.copy()
    df = raw_df.T

    index = []
    cols = []
    for col in df.columns:
        name = col.split(' - ')[1]
        cols.append(name)
    df.columns = cols

    df.index = pd.to_datetime(df.index).to_period('M')

    return df

df = clean(raw_df)
df

,MINISTERIO DE EDUCACIÓN,ADMINISTRACION DE DEUDA PUBLICA Y ACTIVOS FINANCIEROS,MINISTERIO DE SALUD PÚBLICA Y ASISTENCIA SOCIAL,ADMINISTACION DE OBLIGACIONES DEL TESORO NACIONAL,PRESIDENCIA DE LA REPUBLICA,MINISTERIO DE INTERIOR Y POLICIA,MINISTERIO DE OBRAS PUBLICAS Y COMUNICACIONES,MINISTERIO DE DEFENSA,MINISTERIO DE HACIENDA,MINISTERIO DE EDUCACION SUPERIOR CIENCIA Y TECNOLOGIA,...,SENADO DE LA REPUBLICA,CÁMARA DE CUENTAS,TRIBUNAL CONSTITUCIONAL,MINISTERIO DE ENERGIA Y MINAS,MINISTERIO DE ADMINISTRACION PUBLICA,MINISTERIO DE LA MUJER,MINISTERIO DE LA JUVENTUD,"MINISTERIO DE LA VIVIENDA, HABITAT Y EDIFICACIONES (MIVHED)",TRIBUNAL SUPERIOR ELECTORAL (TSE),DEFENSOR DEL PUEBLO
2004-01,767.431394,2957.048138,441.501377,1522.415598,362.220320,652.003699,61.212292,379.487534,117.150522,115.394787,...,NaN,10.833333,NaN,NaN,NaN,6.421497,5.144062,NaN,NaN,NaN
2004-02,1283.158893,363.081423,1114.511859,1392.365401,1928.116568,331.174289,121.288006,401.215430,161.325115,119.969762,...,NaN,10.833333,NaN,NaN,NaN,8.601356,6.156662,NaN,NaN,NaN
2004-03,761.393822,1495.604579,620.576890,1722.824601,752.496154,1198.357116,211.422295,460.689713,186.615808,122.484633,...,NaN,10.833334,NaN,NaN,NaN,15.084596,17.294818,NaN,NaN,NaN
2004-04,977.228918,1423.852384,783.188067,1550.421226,2835.405955,795.622783,268.587660,497.840682,248.194468,124.737051,...,NaN,10.833333,NaN,NaN,NaN,13.509925,6.629613,NaN,NaN,NaN
2004-05,1045.296284,0.000000,669.940108,1847.898740,1667.379770,886.941661,291.520428,506.063817,173.627554,122.558712,...,NaN,10.833333,NaN,NaN,NaN,9.652545,8.065232,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2022-01,13046.961683,32591.870267,6580.286207,6298.078271,4996.649184,3393.012747,771.161549,2502.888647,1280.822756,831.437209,...,219.648243,94.593731,131.280973,71.162298,77.850770,68.138999,14.598395,320.050740,75.156794,15.525817
2022-02,17651.401885,13004.240149,9408.916033,7497.604893,5846.971695,3671.233224,2292.269967,2923.434915,1439.923821,1208.195773,...,219.648243,135.993184,131.280979,165.664163,92.504293,68.141596,23.825447,794.982172,75.156794,17.803155
2022-03,17074.320966,15142.799726,11446.616584,7533.673448,6055.164319,3771.267868,2644.279395,3069.225242,1555.993971,1250.827179,...,219.648243,123.221389,131.280975,143.217349,103.550673,86.237894,83.142541,775.618666,75.156794,20.156249
2022-04,17934.264403,4350.896030,8734.230454,10362.093483,6010.677870,3875.570851,2106.287016,3414.650089,1536.917768,1128.365798,...,219.648243,124.489479,131.280975,142.459411,115.412960,111.322752,34.844243,487.147903,75.156794,25.394027


In [25]:
# Melt
def melt(df):
    df = df.copy()
    cols = df.columns
    df.reset_index(inplace=True)
    df.rename(columns={'index':'Fecha'}, inplace=True)
    
    df = pd.melt(
        df,
        id_vars = ['Fecha'],
        value_vars = cols,
        var_name='Organizacion',
        value_name='Valor'
    )

    df.set_index('Fecha', drop=True, inplace=True)

    return df
df = melt(df)
df

,Organizacion,Valor
Fecha,,
2004-01,MINISTERIO DE EDUCACIÓN,767.431394
2004-02,MINISTERIO DE EDUCACIÓN,1283.158893
2004-03,MINISTERIO DE EDUCACIÓN,761.393822
2004-04,MINISTERIO DE EDUCACIÓN,977.228918
2004-05,MINISTERIO DE EDUCACIÓN,1045.296284
...,...,...
2022-01,DEFENSOR DEL PUEBLO,15.525817
2022-02,DEFENSOR DEL PUEBLO,17.803155
2022-03,DEFENSOR DEL PUEBLO,20.156249


In [26]:
fig = px.area(df, x=df.index.strftime("%Y-%m"), y='Valor', color='Organizacion').show()

In [27]:
def market_share(df):
    df = df.copy()

    for i in df.index:
        total = df.loc[i].sum()['Valor']
        df.loc[i,'MS'] = df.loc[i]['Valor'] / total

    return df
df = market_share(df)
df

,Organizacion,Valor,MS
Fecha,,,
2004-01,MINISTERIO DE EDUCACIÓN,767.431394,0.093265
2004-02,MINISTERIO DE EDUCACIÓN,1283.158893,0.153802
2004-03,MINISTERIO DE EDUCACIÓN,761.393822,0.085321
2004-04,MINISTERIO DE EDUCACIÓN,977.228918,0.086344
2004-05,MINISTERIO DE EDUCACIÓN,1045.296284,0.123328
...,...,...,...
2022-01,DEFENSOR DEL PUEBLO,15.525817,0.000198
2022-02,DEFENSOR DEL PUEBLO,17.803155,0.000242
2022-03,DEFENSOR DEL PUEBLO,20.156249,0.000257


In [138]:
def top_orgs(df, n=10):
    df = df.copy()

    max_date = max(df.index)
    max_df = df.groupby(by=['Organizacion']).sum()['Valor']
    max_df = max_df.sort_values(ascending=False)
    max_df = max_df.reset_index()
    top = max_df[:n]
    orgs = list(top['Organizacion'])

    top_orgs = df[df['Organizacion'].isin(orgs)]
    bottom_orgs = df[~df['Organizacion'].isin(orgs)]
    bottom = bottom_orgs.reset_index()
    bottom = bottom_orgs.groupby(by=['Fecha']).sum()
    bottom['Organizacion'] = 'Demas'
    df = pd.concat([top_orgs,bottom])

    # df.sort_values(by=['Valor'])
    
    return df

top = top_orgs(df)
top


,Organizacion,Valor,MS
Fecha,,,
2004-01,MINISTERIO DE EDUCACIÓN,767.431394,0.093265
2004-02,MINISTERIO DE EDUCACIÓN,1283.158893,0.153802
2004-03,MINISTERIO DE EDUCACIÓN,761.393822,0.085321
2004-04,MINISTERIO DE EDUCACIÓN,977.228918,0.086344
2004-05,MINISTERIO DE EDUCACIÓN,1045.296284,0.123328
...,...,...,...
2022-01,Demas,6225.114216,0.079282
2022-02,Demas,8516.886542,0.115937
2022-03,Demas,8913.732570,0.113612


In [140]:
def roll(df, n=6):
    df = df.copy()
    # df = df.sort_index()
    all_df = pd.DataFrame()

    for org in df['Organizacion'].unique():
        temp = df[df['Organizacion'] == org]
        temp = temp[['Valor','MS']].rolling(6).mean()
        temp['Organizacion'] = org
        all_df = pd.concat([all_df, temp], ignore_index=False)
        # print(org)
        # print(temp)
        # print('#################')
    all_df.dropna(inplace=True)
    return all_df

rolled = roll(top, 6)
rolled

,Valor,MS,Organizacion
Fecha,,,
2004-06,951.625141,0.107717,MINISTERIO DE EDUCACIÓN
2004-07,993.416862,0.110238,MINISTERIO DE EDUCACIÓN
2004-08,908.932407,0.100423,MINISTERIO DE EDUCACIÓN
2004-09,921.248204,0.103156,MINISTERIO DE EDUCACIÓN
2004-10,881.183186,0.106849,MINISTERIO DE EDUCACIÓN
...,...,...,...
2022-01,12189.866056,0.120696,Demas
2022-02,12137.462929,0.118466,Demas
2022-03,12095.010412,0.118240,Demas


In [141]:
def area_chart(df):
    df = df.copy().dropna()
    fig = px.area(df, x=df.index.strftime("%Y-%m"), y='MS', color='Organizacion', title='Gasto como % por Organizacion')
    fig.update_layout(legend_traceorder="reversed",height=800)
    fig.show()
area_chart(rolled)

In [142]:
def bar_chart(df):
    df = df.copy().dropna()
    fig = px.area(df, x=df.index.strftime("%Y-%m"), y='Valor', color='Organizacion', title='Gasto como % por Organizacion')
    fig.update_layout(legend_traceorder="reversed",height=800)
    fig.show()
bar_chart(rolled)

In [143]:
def line_MS(df):
    fig = px.line(df,x=df.index.strftime('%Y-%m'),y='MS',color='Organizacion',title='Gasto como % por Organizacion')
    fig.update_layout(legend_traceorder="reversed",height=800)
    fig.show()
line_MS(rolled)

In [48]:
top = df[df['Organizacion'].isin(cols)]
bottom = df[~df['Organizacion'].isin(cols)]


,Organizacion,Valor,MS
Fecha,,,
2004-01,MINISTERIO DE EDUCACION SUPERIOR CIENCIA Y TEC...,115.394787,0.014024
2004-02,MINISTERIO DE EDUCACION SUPERIOR CIENCIA Y TEC...,119.969762,0.014380
2004-03,MINISTERIO DE EDUCACION SUPERIOR CIENCIA Y TEC...,122.484633,0.013726
2004-04,MINISTERIO DE EDUCACION SUPERIOR CIENCIA Y TEC...,124.737051,0.011021
2004-05,MINISTERIO DE EDUCACION SUPERIOR CIENCIA Y TEC...,122.558712,0.014460
...,...,...,...
2022-01,DEFENSOR DEL PUEBLO,15.525817,0.000198
2022-02,DEFENSOR DEL PUEBLO,17.803155,0.000242
2022-03,DEFENSOR DEL PUEBLO,20.156249,0.000257
